In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import pickle

# 1. Leer el dataset final
df = pd.read_csv('data/data_FINAL2.csv')

# Convertir la variable objetivo "Precio" a numérico y eliminar registros sin precio
df['Precio'] = pd.to_numeric(df['Precio'], errors='coerce')
df = df.dropna(subset=['Precio'])

# 2. Seleccionar la variable objetivo y las variables predictoras
y = df['Precio']
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']
X = df[features]

# 3. Aplicar Label Encoding a las columnas categóricas y guardar los encoders
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']
X_encoded = X.copy()
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    encoders[col] = le  # Guardamos el encoder para la columna

# Guardar los encoders en un archivo para usarlos en la predicción
with open('encoders/encoders_xgboost.pkl', 'wb') as f:
    pickle.dump(encoders, f)

# 4. Dividir el dataset en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=89)

# 5. Definir el modelo XGBoost y configurar la búsqueda de hiperparámetros
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=82)

param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='r2', 
                           n_jobs=-1, 
                           verbose=1)

grid_search.fit(X_train, y_train)

print("\n========\n")
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("\n========\n")

# Utilizar el mejor estimador encontrado
best_xgb = grid_search.best_estimator_

# 6. Evaluar el modelo en el conjunto de prueba
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n========\n")
print("Evaluación del modelo XGBoost:")
print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)
print("\n========\n")

# 7. Guardar el modelo entrenado en un archivo
joblib.dump(best_xgb, 'models/modelo_xgboost.pkl')
print("El modelo se ha guardado en 'modelo_xgboost.pkl'")

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


Mejores parámetros encontrados: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 700, 'subsample': 0.8}




Evaluación del modelo XGBoost:
MSE: 14966760.078804273
MAE: 2569.582783203125
R²: 0.8516087470577309


El modelo se ha guardado en 'modelo_xgboost.pkl'
